## Here we will be making our model

we will begin by importing required libraries

In [155]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


now that we have our libraries imported, we will first define some global variables

In [156]:
ALPHA = 0.0001
ITERATIONS = 3000
W = np.zeros(8)
B = 0

now we will first import and feature scale data using z-scale normalisation

In [157]:
def getData():
    df = pd.read_csv('CleanDataNP.csv')
    X = df.to_numpy()[:,0:-1]
    Y = df.to_numpy()[:,-1]
    mux = np.mean(X, axis=0)
    muy = np.mean(Y, axis=0)
    # print(np.shape(mux))
    sigx = np.std(X, axis=0)
    sigy = np.std(Y, axis=0)
    return (X - mux)/sigx, (Y-muy)/sigy, mux, muy, sigx, sigy

rX, rY, muX, muY, stdX, stdY = getData()

before moving further, let's first divide our data into 3 parts-  
1. Training set (X, Y) - 60%
2. Dev set (dX, dY) - 20%
3. Test set (tX. tY) - 20%

In [158]:
X, Y = rX[:5200, :], rY[:5200]
dX, dY = rX[5200:6950, :], rY[5200:6950]
tX, tY = rX[6950:, :], rY[6950:]
print(np.shape(X), np.shape(Y), np.shape(dX), np.shape(dY), np.shape(tX), np.shape(tY))

(5200, 8) (5200,) (1750, 8) (1750,) (1768, 8) (1768,)


now we have the data which is 100% ready to be fed into model.  
so now let's make a function that calculate cost function

In [159]:
def calculateCost(x, y, w, b):
    cost = 0
    for i in range(len(y)):
        cost += (((np.dot(w, x[i]) + b) - y[i])**2)
    return (cost/(2*len(y)))
print(calculateCost(X, Y, W, B))

0.5388097996627864


let's also make a function to calculate MOE (mean obsolute error)

In [160]:
def calculateMOE(x, y, w, b):
    moe = 0
    for i in range(len(y)):
        moe += abs(np.dot(w, x[i]) + b - y[i])
    return (moe/(len(y)))
print(calculateMOE(X, Y, W, B))

0.5976416438749139


Now we will be making a function to calculate gradient for our gradient discent algo

In [161]:
def getGradient(x, y, w, b):
    grw = 0
    grb = 0
    for i in range(len(y)):
        grw += ((np.dot(w, x[i]) + b) - y[i])*x[i]
        grb += ((np.dot(w, x[i]) + b) - y[i])
    return (grw/(len(y))), (grb/(len(y)))

now perhaps we are ready to iterate and get the real values for w and b

In [162]:
def getWB(x, y, w, b):
    global W, B, tX, tY
    for i in range(ITERATIONS):
        grw, grb = getGradient(x, y, w, b)
        tw = W - ALPHA*grw
        tb = B - ALPHA*grb
        W, B = tw, tb
        if i%50 == 0:
            print("At iteration:", i, "Cost:", calculateCost(X, Y, W, B), "MOE:", calculateMOE(X, Y, W, B), "|| Unseen MOE:", calculateMOE(tX, tY, W, B))
getWB(X, Y, W, B)
print(W, B)

At iteration: 0 Cost: 0.5386897773563082 MOE: 0.5975305156106249 || Unseen MOE: 0.5606573651042349
At iteration: 50 Cost: 0.5327257665109255 MOE: 0.5919741519150954 || Unseen MOE: 0.5551529029489569
At iteration: 100 Cost: 0.526834509545411 MOE: 0.5864248457278262 || Unseen MOE: 0.549650227435265
At iteration: 150 Cost: 0.5210160064597581 MOE: 0.5809016233299745 || Unseen MOE: 0.5441578632156875
At iteration: 200 Cost: 0.5152702572539634 MOE: 0.5754445921269569 || Unseen MOE: 0.5387315403606351
At iteration: 250 Cost: 0.5095972619280391 MOE: 0.5700033121744081 || Unseen MOE: 0.5333064401950341
At iteration: 300 Cost: 0.5039970204819774 MOE: 0.5645856646013844 || Unseen MOE: 0.5278827418050335
At iteration: 350 Cost: 0.49846953291577495 MOE: 0.5592035118364475 || Unseen MOE: 0.5224902087375835
At iteration: 400 Cost: 0.4930147992294372 MOE: 0.5538713754089928 || Unseen MOE: 0.51714257967951
At iteration: 450 Cost: 0.4876328194229633 MOE: 0.5485844746162647 || Unseen MOE: 0.5118360803154

In [163]:
prices = np.dot(W, dX[0]) + B
print(prices*stdY + muY, dY[0]*stdY + muY)

3353611.798859672 1098000.0


## So now our model isn't learning...
33L predicted vs 10.9L real is very very wrong!!!

now we will be doing some feature engineering and then using regularization.